# Asyncio High-Level Review
- Writing concurrent code using the async/await syntax

- Coroutines and Tasks
    - Thing we strive to make async/await
    - Needs an entry-point: asyncio.run() -> top-level main

- Awaitable: things with an await dunder
    - Three types: coroutines, Tasks, Futures

- Coroutine definition
    1. A function: async def signature
    2. Object: returned by calling a coroutine function

- Future: low-level awaitable object which represents an eventual result of an async op

## Methods of Interest
    - Creating Tasks
        1. asyncio.create_task(coro, *, name=None, context=None)
            - If a reference isn't saved, a fire and forget could lose the task
        2. asyncio.TaskGroup -> context manager way
            - Downside: waits for all tasks in the group to finish
            - *Downside/Plus: early exits don't raise the asyncio.CancelledError exception, tons of exceptions to this rule!!
                - Due to this, managing through create_task is better/more granular!
    - Running Tasks
        1. asyncio.run(coro, *, debug=None, loop_factory=None)
        2. class asyncio.Runner(*, debug=None, loop_factory=None)
            - Ways to operate
                - run(coro, *, context=None)
                - close()
                - get_loop()
            - Notes
                - Runner uses the lazy initialization strategy, its constructor doesn’t initialize underlying low-level structures
                - Embedded loop and context are created at the with body entering or the first call of run() or get_loop()
                - Installs a custom signal.SIGINT handler before any user code is executed and removes it when exiting from the function
        3. [Within an asyncio function] awaitable asyncio.gather(*aws, return_exceptions=False)
            - Same as creating tasks number 2: dont do it. This way you don't have to use shielding

    - Waiting
        1. The asyncio.timeout() context manager is what transforms the asyncio.CancelledError into a TimeoutError 
        2. async asyncio.wait_for(aw, timeout)
        3. asyncio.as_completed(aws, *, timeout=None) -> returns an iterable
            - Async iterator supplied
            - Avoid if possible as the task completion makes it difficult 

    - Introspection
        1. asyncio.current_task(loop=None) -> returns the currently running Task instance, or None
        2. asyncio.iscoroutine(obj) -> returns bool

## High-Level Task Object
- class asyncio.Task(coro, *, loop=None, name=None, context=None, eager_start=False)
- A Future-like object that runs a Python coroutine. Not threadsafe!
- Used to run coroutines in event loops
    - If a coroutine awaits on a Future, the Task suspends the execution of the coroutine and waits for the completion of the Future
    - When the Future is done, the execution of the wrapped coroutine resumes
- Special low-level awaitable obj that represents an eventual result of an async op
    - Needed for callback-based code used with async/await
- asyncio.Task inherits from Future all of its APIs except Future.set_result() and Future.set_exception()
- Some methods: done(), result(), get_name(), set_name(value), cancel(msg=None)

## Event Loop
- Event loops use cooperative scheduling (on Task at a time)
- Use the high-level asyncio.create_task() function to create Tasks, or the low-level loop.create_task() or ensure_future() functions.
- add_done_callback(callback, *, context=None)

## Future Addition
- Call Graph Introspection (3.14)
- asyncio.to_thread(...)
- asyncio.run_coroutine_threadsafe(...)
- Low-Level Futures and Event Loop (good for tail tracking of a file)


In [11]:
import asyncio

from dataclasses import dataclass

## Runners
- Runner
    - Uses the lazy initialization strategy
    - Context manager that simplifies multiple async function calls in the same context
    - loop_factory could be used for overriding the loop creation. It is the responsibility of the loop_factory to set the created loop as the current one. By default asyncio.new_event_loop() is used and set as current event loop with asyncio.set_event_loop() if loop_factory is None

    - Methods
        1. run(coro, *, context=None) -> returns coroutine's result or raises its exception
        2. close()
        3. get_loop()

- Dont rely on signal.SIGINT: not propagated into the runner
    - Would need to register a custom signal.SIGINT handler

In [8]:
async def ex():
    await asyncio.sleep(1)
    return 'DONE'

In [9]:
try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # 'RuntimeError: There is no current event loop...'
    loop = None

if loop and loop.is_running():
    print('Async event loop already running. Adding coroutine to the event loop.')
    tsk = loop.create_task(ex())
    tsk.add_done_callback(
        lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
else:
    print('Starting new event loop')
    
    with asyncio.Runner() as runner:
        runner.run(ex())

Async event loop already running. Adding coroutine to the event loop.


Task done with result=DONE  << return val of main()


## Coroutines and Tasks 

In [8]:
import asyncio
import time

from asyncio import TaskGroup

In [ ]:
ipv4_connect = asyncio.create_task(asyncio.open_connection("127.0.0.1", 80))
ipv6_connect = asyncio.create_task(asyncio.open_connection("::1", 80))
tasks = [ipv4_connect, ipv6_connect]

async for earliest_connect in asyncio.as_completed(tasks):
    # earliest_connect is done. The result can be obtained by
    # awaiting it or calling earliest_connect.result()
    reader, writer = await earliest_connect

    if earliest_connect is ipv6_connect:
        print("IPv6 connection established.")
    else:
        print("IPv4 connection established.")

In [ ]:
async def main():
    await asyncio.sleep(1)
    print('hello')

with asyncio.Runner() as runner:
    runner.run(main())

In [ ]:
async def main():
    try:
        async with asyncio.timeout(10):
            await long_running_task()
    except TimeoutError:
        print("The long operation timed out, but we've handled it.")

    try:
        await asyncio.wait_for(eternity(), timeout=1.6)
    except TimeoutError:
        print('timeout!')

    print("This statement will run regardless.")

In [ ]:
async def display_date():
    loop = asyncio.get_running_loop()
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

In [9]:
async def job(task_id, sleep_time):
    print(f'Task {task_id}: start')
    await asyncio.sleep(sleep_time)
    print(f'Task {task_id}: done')

In [10]:
async def main():
    async with asyncio.TaskGroup() as tg:
        task1 = tg.create_task(
            job(1, 1))

        task2 = tg.create_task(
            job(2, 2))

        print(f"started at {time.strftime('%X')}")

    # The await is implicit when the context manager exits.

    print(f"finished at {time.strftime('%X')}")

In [11]:
class TerminateTaskGroup(Exception):
    """Exception raised to terminate a task group."""

async def force_terminate_task_group():
    """Used to force termination of a task group."""
    raise TerminateTaskGroup()

async def main():
    try:
        async with TaskGroup() as group:
            # spawn some tasks
            group.create_task(job(1, 0.5))
            group.create_task(job(2, 1.5))
            # sleep for 1 second
            await asyncio.sleep(1)
            # add an exception-raising task to force the group to terminate
            group.create_task(force_terminate_task_group())
    except* TerminateTaskGroup:
        pass

# Usually asyncio.run(main()) required but not in a notebook
main()

<coroutine object main at 0x75507c593d80>

## Subprocesses

In [12]:
from typing import Optional

@dataclass
class Shell_Runner:
    cmd: str
    name: Optional[str]
    return_code: str=None
    stdout: str=None
    stderr: str=None

    def __post_init__(self):
        # Run enterprise cmd check per user before run -> ABAC security
        pass 

    def __str__(self) -> str:
        builder = f'{self.name}: ' if self.name else ''

        return f'{builder}\n\tStdout: {self.stdout}\n\t{self.stderr}'

In [ ]:
from typing import Callable, Concatenate, Tuple

# This enable osquery updates to kwargs depending on the user
type ABAC_Shell_Check[**P, T: Shell_Runner, NewTypedDict] = Callable[Concatenate[T, P], NewTypedDict]

async def enterprise_check[T: Shell_Runner, S: (str), NewTypedDict](codename: S, fxn_ptr: ABAC_Shell_Check=None) -> NewTypedDict:
    async def inner(shell_obj: T, **kwargs) -> NewTypedDict:
        # Run enterprise check here
        rest_call = ... # awaitable with kwargs
        shell_obj = await run_shell(shell_obj)

        return fxn_ptr(rest_call, shell_obj, **kwargs)
    
    # DO things here with the codename
    return inner

# **kwargs is for expansion and could be TypedDict later
## Also, **kwargs could be part of a TypeExpr for protocol-based state transition flows within an OS/SELinux-defined box
@enterprise_check('blue')
async def run_shell[T: Shell_Runner](shell_obj: T, **kwargs) -> T:
    proc = await asyncio.create_subprocess_shell(
        shell_obj.cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE)

    stdout, stderr = await proc.communicate()

    shell_obj.return_code = proc.returncode
    
    if stdout:
        shell_obj.stdout = stdout.decode()
    if stderr:
        shell_obj.stderr = stderr.decode()

    return shell_obj

In [ ]:
cmds = [['ls -l', 'simple file viewer'], ['cat /etc/passwd', 'looking at passwords']]

shell_objs = [Shell_Runner(*info) for info in cmds)]

async for ret_obj in asyncio.gather(map(run_shell, shell_objs)):
    print(ret_obj)

## Future
- Methods:
    - result()
        - If done: set_result() or set_exception() called
        - If cancelled: raises a CancelledError exception
        - If result isnt available yet, raised an InvalidStateError exception
    - done() -> bool
    - cancelled -> bool
    - add_done_callback(callback, *, context=None)
        - Adds a callback to be run when the Future is done
    - exception()
        - Return the exception that was set on this Future
    - cancel(msg=None)
        - Cancel the Future and [Important] schedule callbacks
    - get_loop() -> bound event loop

In [17]:
async def set_after(fut, delay, value):
    await asyncio.sleep(delay)

    fut.set_result(value)

async def main():
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # 'RuntimeError: There is no current event loop...'
        return

    fut = loop.create_future()

    loop.create_task(
        set_after(fut, 1, '... world'))

    print('hello ...')

    print(await fut)

await main()

hello ...
... world


## Event Loop
- Core of every asyncio application

- Obtaining the Event Loop
    1. get_running_loop() -> returns current OS thread loop
    2. get_event_loop(): avoid this due to complex behavior
    3. set_event_loop(loop)
        - Sets for the current OS thread
    4. new_event_loop()

- Methods
    1. run_until_complete(future)
    2. run_forever()
        - Run until stop() is called
    3. is_running() -> bool
    4. [Important] shutdown_default_executor(timeout=None)
        - Close using the default executor and wait for it to join all of the threads in the ThreadPoolExecutor; default of None allows unlimited runtime
        - If the timeout is reached, a RuntimeWarning is emitted and the default executor is terminated without waiting for its threads to finish joining.
        - Do not call with asyncio.run()

- Scheduling Callbacks
    - Can be used to replace cron files

### Handling Asyncio Exceptions
- https://superfastpython.com/asyncio-event-loop-exception-handler/#Example_Default_Exception_Handler

In [24]:
import asyncio
 
# define an exception handler
def exception_handler(loop, context):
    # get the exception
    ex = context['exception']
    # log details
    print(f'Got exception -> {ex}')
 
# task that does work
async def work():
    # block for a moment
    await asyncio.sleep(1)
    # raise a problem
    raise Exception('Something Bad Happened')
 
# main coroutine
async def main():
    # get the event loop
    loop = asyncio.get_running_loop()
    # set the exception handler
    loop.set_exception_handler(exception_handler)
    # report a message
    print('Starting')
    # run the task
    _ = asyncio.create_task(work())
    # block for a moment
    await asyncio.sleep(2)
    # report a message
    print('Done')
 
# start the event loop
await main()

Starting
Done
Got exception -> Something Bad Happened


### Separate Event Loop Per Thread
- https://superfastpython.com/asyncio-multiple-event-loops/

In [25]:
import threading
import asyncio
import time
 
# main coroutine for the asyncio program
async def main_coroutine(name):
    # report a message
    print(f'{name} coroutine running...', flush=True)
    # get the loop for this thread
    loop = asyncio.get_running_loop()
    # report details of the current event loop
    print(f'{name}: {id(loop)} - {loop}', flush=True)
    for i in range(6):
        # report a message
        print(f'{name}: running...', flush=True)
        # suspend a moment
        await asyncio.sleep(0.5)
    # report a final message
    print(f'{name} done', flush=True)
 
# create and start the threads
threads = list()
for i in range(5):
    # create the thread
    thread = threading.Thread(target=asyncio.run, args=(main_coroutine(f'Loop{i}'),))
    # store
    threads.append(thread)
    # start the thread
    thread.start()
# wait for all threads to be done
for thread in threads:
    thread.join()

Loop0 coroutine running...
Loop1 coroutine running...
Loop0: 127645558181824 - <_UnixSelectorEventLoop running=True closed=False debug=False>
Loop1: 127645558182432 - <_UnixSelectorEventLoop running=True closed=False debug=False>
Loop3 coroutine running...
Loop0: running...Loop3: 127645558182736 - <_UnixSelectorEventLoop running=True closed=False debug=False>
Loop1: running...
Loop2 coroutine running...

Loop3: running...
Loop2: 127645558181520 - <_UnixSelectorEventLoop running=True closed=False debug=False>
Loop4 coroutine running...
Loop2: running...
Loop4: 127645558183040 - <_UnixSelectorEventLoop running=True closed=False debug=False>
Loop4: running...
Loop1: running...Loop0: running...

Loop3: running...
Loop2: running...
Loop4: running...
Loop0: running...
Loop1: running...
Loop3: running...
Loop2: running...
Loop4: running...
Loop0: running...
Loop1: running...
Loop3: running...Loop2: running...

Loop4: running...
Loop0: running...Loop1: running...

Loop3: running...Loop2: runni

### Example: Threads Within Awaitable Handler
- Cooperative multitasking or coroutine-based concurrency

In [20]:
import asyncio
import concurrent.futures

def blocking_io():
    # File operations (such as logging) can block the
    # event loop: run them in a thread pool.
    with open('/dev/urandom', 'rb') as f:
        return f.read(10)

def cpu_bound():
    # CPU-bound operations will block the event loop:
    # in general it is preferable to run them in a
    # process pool.
    return sum(i * i for i in range(10 ** 7))

async def main():
    loop = asyncio.get_running_loop()

    ## Options:

    # 1. Run in the default loop's executor:
    result = await loop.run_in_executor(
        None, blocking_io)
    print('default thread pool', result)

    # 2. Run in a custom thread pool:
    with concurrent.futures.ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(
            pool, blocking_io)
        print('custom thread pool', result)

    # 3. Run in a custom process pool:
    with concurrent.futures.ProcessPoolExecutor() as pool:
        result = await loop.run_in_executor(
            pool, cpu_bound)
        print('custom process pool', result)

await main()

default thread pool b'\x98\xc3\x0e\xdc\xd3\xd3\xbb\xad\x7f\x14'
custom thread pool b'l\xaa!\xd8+\xe1-]\xee\xee'
custom process pool 333333283333335000000


### The Event Loop: Third-Party Analysis
- Cooperative multitasking or coroutine-based concurrency
- Examples from: https://codilime.com/blog/how-fit-triangles-into-squares-run-blocking-functions-event-loop/

#### Ex 1
- By default, the maximum number of worker processes is set to the number of CPU cores available.

- Only one problem: "Almost all asyncio objects are not thread-safe, which is typically not a problem unless there is code that works with them from outside of a Task or a callback." 
    - When asyncio spawns a thread pool, each of the threads is a different OS thread. Therefore one task running in the pool can't safely access a shared object from another task. 

In [21]:
import asyncio
import time
import concurrent.futures


async def non_blocking_operation() -> None:
    # Simulate a non-blocking operation
    # using non-blocking asyncio.sleep
    await asyncio.sleep(3)


def blocking_cpu_operation() -> None:
    # Simulate a long-running CPU-bound operation
    # using a busy-wait loop
    x = 0
    for i in range(10**7):
        x += i


async def main() -> None:
    # Get the default event loop.
    # It will be created if it doesn't exist yet.
    loop = asyncio.get_running_loop()

    print("Starting process pool executor blocking CPU operation")
    # Create io_bound_operation in the event loop
    io_bound_operation = non_blocking_operation()
    with concurrent.futures.ProcessPoolExecutor() as pool:
        # Create cpu_bound_operation in the process pool executor
        cpu_bound_operation = loop.run_in_executor(pool, blocking_cpu_operation)

    start = time.perf_counter()
    await asyncio.gather(cpu_bound_operation, io_bound_operation)
    stop = time.perf_counter()
    print(f"Done in {stop - start:.2f} seconds.")

await main()

Starting process pool executor blocking CPU operation
Done in 3.00 seconds.


#### Ex 2

In [ ]:
import asyncio
import io
import concurrent.futures
import threading
import time

def load_data(resource, f, lock):
   data = f.read()
   with lock:
       resource["data"] = data

def process_data(resource, lock):
   while True:
       with lock:
           if resource["data"] is None:  # wait for data to be loaded
               print("Waiting for data to process")
               continue
           if resource["data"]:
               print("Received data, processing")
               time.sleep(1)  # simulate processing
           break

async def main():
   # prepare a file-like object that will simulate a stream of data
   f = io.StringIO("I can fit triangles into squares")
   # prepare a resource that will be shared between tasks
   resource = {"type": "article", "data": None}
   # create a lock to synchronize access to the shared resource
   lock = threading.Lock()

   loop = asyncio.get_event_loop()
   with concurrent.futures.ThreadPoolExecutor() as pool:
       # spawn process data task that will wait for data to process
       loop.run_in_executor(pool, process_data, resource, lock)
       # spawn two tasks that will load data concurrently
       loop.run_in_executor(pool, load_data, resource, f, lock)
       loop.run_in_executor(pool, load_data, resource, f, lock)

#### ChatGPT Generated Examples

In [ ]:
import threading
import asyncio
import time
from dataclasses import dataclass, field
from typing import List, Optional, Union

@dataclass
class ExecutionResult:
    name: str
    log: List[str] = field(default_factory=list)
    result: Union[str, Exception] = "Success"

def exception_handler(loop, context):
    """Handle exceptions in asyncio event loops."""
    ex = context.get('exception')
    thread_name = threading.current_thread().name
    print(f'Exception in {thread_name}: {ex}', flush=True)
    if thread_name in results:
        results[thread_name].result = ex

async def main_coroutine(name):
    """Main coroutine that runs in an asyncio event loop."""
    loop = asyncio.get_running_loop()
    loop.set_exception_handler(exception_handler)
    thread_name = threading.current_thread().name
    results[thread_name] = ExecutionResult(name=name)
    
    results[thread_name].log.append(f'{name}: Loop ID {id(loop)} running in thread {thread_name}')
    print(results[thread_name].log[-1], flush=True)
    
    for i in range(3):
        try:
            results[thread_name].log.append(f'{name}: running iteration {i}')
            print(results[thread_name].log[-1], flush=True)
            await asyncio.sleep(0.5)

            if i == 2:
                raise Exception('fake exception at i == 2 within the loop')
        except Exception as e:
            results[thread_name].result = e
        else:
            results[thread_name].result = "Success"

def run_threaded_loop(name):
    """Isolated function to run the asyncio loop in a separate thread."""
    thread = threading.Thread(target=asyncio.run, args=(main_coroutine(name),), name=name)
    thread.start()
    return thread

# Store execution results
results = {}

# Start threads running their own event loops
threads = [run_threaded_loop(f'Loop{i}') for i in range(3)]

# Wait for all threads to complete
for thread in threads:
    thread.join()

print('All threads finished execution.')

# Print stored results
for thread_name, result in results.items():
    print(f'\nResults for {thread_name}:')
    for log_entry in result.log:
        print(log_entry)
    if isinstance(result.result, Exception):
        print(f'Exception occurred: {result.result}')
    else:
        print('Execution successful.')

Loop0: Loop ID 127645558181520 running in thread Loop0
Loop1: Loop ID 127645558178480 running in thread Loop1
Loop2: Loop ID 127645558179696 running in thread Loop2
Loop2: running iteration 0
Loop1: running iteration 0
Loop0: running iteration 0
Loop0: running iteration 1
Loop2: running iteration 1
Loop1: running iteration 1
Loop1: running iteration 2
Loop0: running iteration 2
Loop2: running iteration 2
All threads finished execution.

Results for Loop0:
Loop0: Loop ID 127645558181520 running in thread Loop0
Loop0: running iteration 0
Loop0: running iteration 1
Loop0: running iteration 2
Exception occurred: fake exception

Results for Loop1:
Loop1: Loop ID 127645558178480 running in thread Loop1
Loop1: running iteration 0
Loop1: running iteration 1
Loop1: running iteration 2
Exception occurred: fake exception

Results for Loop2:
Loop2: Loop ID 127645558179696 running in thread Loop2
Loop2: running iteration 0
Loop2: running iteration 1
Loop2: running iteration 2
Exception occurred: fa

In [ ]:

import threading
import asyncio
import time
from dataclasses import dataclass, field
from typing import List, Optional, Union, ClassVar, Dict, Final, Tuple

class ThreadManagerMeta(type):
    """Metaclass to initialize thread loops within __new__."""
    def __new__(mcs, name, bases, dct):
        cls = super().__new__(mcs, name, bases, dct)
        # Would be better if 3 was instead a variable defined by the first instance -> __init__(workers: int=3)
        cls.threads = (
            (asyncio.new_event_loop(), None) for _ in range(3)
        )
        return cls

@dataclass
class ExecutionResult:
    name: str
    log: List[str] = field(default_factory=list)
    result: Union[str, Exception] = "Success"

type Isolated_EventLoop = Tuple[asyncio.AbstractEventLoop, threading.Thread]
type Threading_EventLoops = Tuple[Isolated_EventLoop, ...]

class AsyncThreadManager(metaclass=ThreadManagerMeta):
    """Manages multiple threads running their own asyncio loops."""
    results: ClassVar[Dict[str, ExecutionResult]] = {}
    thread: ClassVar[Threading_EventLoops]
    
    @staticmethod
    def exception_handler(loop, context):
        """Handle exceptions in asyncio event loops."""
        ex = context.get('exception')
        thread_name = threading.current_thread().name
        print(f'Exception in {thread_name}: {ex}', flush=True)
        if thread_name in AsyncThreadManager.results:
            AsyncThreadManager.results[thread_name].result = ex

    @staticmethod
    async def main_coroutine(name):
        """Main coroutine that runs in an asyncio event loop."""
        loop = asyncio.get_running_loop()
        loop.set_exception_handler(AsyncThreadManager.exception_handler)
        thread_name = threading.current_thread().name
        AsyncThreadManager.results[thread_name] = ExecutionResult(name=name)
        
        AsyncThreadManager.results[thread_name].log.append(
            f'{name}: Loop ID {id(loop)} running in thread {thread_name}'
        )
        print(AsyncThreadManager.results[thread_name].log[-1], flush=True)
        
        try:
            for i in range(3):
                AsyncThreadManager.results[thread_name].log.append(f'{name}: running iteration {i}')
                print(AsyncThreadManager.results[thread_name].log[-1], flush=True)
                await asyncio.sleep(0.5)
        except Exception as e:
            AsyncThreadManager.results[thread_name].result = e
        else:
            AsyncThreadManager.results[thread_name].result = "Success"

    @classmethod
    def run_threaded_loop(cls, name, loop):
        """Run an asyncio loop in a separate thread."""
        def start_loop():
            asyncio.set_event_loop(loop)
            loop.run_until_complete(cls.main_coroutine(name))
        thread = threading.Thread(target=start_loop, name=name)
        thread.start()
        return thread

    @classmethod
    def start_threads(cls):
        """Initialize and start multiple threads."""
        for i, (loop, _) in enumerate(cls.threads):
            cls.threads[i] = (loop, cls.run_threaded_loop(f'Loop{i}', loop))

    @classmethod
    def wait_for_threads(cls):
        """Wait for all threads to complete."""
        for _, thread in cls.threads:
            if thread:
                thread.join()

    @classmethod
    def display_results(cls):
        """Display execution results."""
        print('All threads finished execution.')
        for thread_name, result in cls.results.items():
            print(f'\nResults for {thread_name}:')
            for log_entry in result.log:
                print(log_entry)
            if isinstance(result.result, Exception):
                print(f'Exception occurred: {result.result}')
            else:
                print('Execution successful.')

# Run the thread manager
AsyncThreadManager.start_threads()
AsyncThreadManager.wait_for_threads()
AsyncThreadManager.display_results()